# 1. Working With Larger Databases

In the previous lesson we learned how to use joins to combine data from two tables within a database. In reality, most databases have more than 2 tables, so we'll need strategies to be able to write queries to combine data from 3 or more tables.

In this lesson we'll learn some new techniques to work with the sort of databases that most businesses will use. We'll be working with a modified version of a database called Chinook. The Chinook database contains information about a fictional digital music shop - kind of like a mini-iTunes store.

The Chinook database contains information about the artists, songs, and albums from the music shop, as well as information on the shop's employees, customers, and the customers purchases. This information is contained in eleven tables. Like with our Factbook database, a schema diagram will help us understand the available columns and the structure of the data. Here's a schema diagram for the Chinook database:

![](https://s3.amazonaws.com/dq-content/189/chinook-schema.svg)

Looking at all those tables can be overwhelming at first, but generally speaking we will only need to think about the specific tables that have the data we require and their connections. Before we look at some specific tables, let's take a moment to understand the different parts of the schema diagram.

* Tables names are shown in bold, with the columns in each table listed below.
* Each table has one or more columns with shading, which indicates those columns are a primary key. We'll learn more about primary keys in a later lesson, but for now all you need to know is that each row's primary key must be unique.
* Relationships between tables are shown using lines between the tables. The lines indicate which columns are related. You may notice that at least one 'end' of the relationship will be a primary key.

As you work through this lesson, you may need to refer back to the schema diagram. This is normal, so don't be bothered when that happens. It can also be helpful to write a simple query to check the column names and some example data from any of the tables, for instance:

    SELECT * FROM album LIMIT 3;

# 2. Joining Three Tables

In [2]:
%%capture

%load_ext sql

%sql sqlite:///chinook.db

In [3]:
%%sql

SELECT * FROM media_type LIMIT 5;

 * sqlite:///chinook.db
Done.


media_type_id,name
1,MPEG audio file
2,Protected AAC audio file
3,Protected MPEG-4 video file
4,Purchased AAC audio file
5,AAC audio file


In [4]:
%%sql

SELECT * FROM track LIMIT 5;

 * sqlite:///chinook.db
Done.


track_id,name,album_id,media_type_id,genre_id,composer,milliseconds,bytes,unit_price
1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman",230619,3990994,0.99
4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. Dirkscneider & W. Hoffman",252051,4331779,0.99
5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99


In [5]:
%%sql

SELECT * FROM invoice_line LIMIT 5;

 * sqlite:///chinook.db
Done.


invoice_line_id,invoice_id,track_id,unit_price,quantity
1,1,1158,0.99,1
2,1,1159,0.99,1
3,1,1160,0.99,1
4,1,1161,0.99,1
5,1,1162,0.99,1


In [6]:
%%sql

SELECT il.track_id AS track_id,
        t.name AS track_name,
        mt.name AS track_type,
        il.unit_price AS unit_price,
        il.quantity AS quantity
    FROM invoice_line AS il
INNER JOIN track AS T ON t.track_id = il.track_id
INNER JOIN media_type AS mt ON mt.media_type_id = t.media_type_id
WHERE il.invoice_id = 4;

 * sqlite:///chinook.db
Done.


track_id,track_name,track_type,unit_price,quantity
3448,"Lamentations of Jeremiah, First Set \ Incipit Lamentatio",Protected AAC audio file,0.99,1
2560,Violent Pornography,MPEG audio file,0.99,1
3336,War Pigs,Purchased AAC audio file,0.99,1
829,Let's Get Rocked,MPEG audio file,0.99,1
1872,Attitude,MPEG audio file,0.99,1
748,Dealer,MPEG audio file,0.99,1
1778,You're What's Happening (In The World Today),MPEG audio file,0.99,1
2514,Spoonman,MPEG audio file,0.99,1


# 3. Joining More Than Three Tables

In [7]:
%%sql

SELECT il.track_id AS track_id,
        t.name AS track_name,
        ar.name AS artist_name,
        mt.name AS track_type,
        il.unit_price AS unit_price,
        il.quantity AS quantity
    FROM invoice_line AS il
INNER JOIN track AS T ON t.track_id = il.track_id
INNER JOIN media_type AS mt ON mt.media_type_id = t.media_type_id
INNER JOIN album AS al ON al.album_id = t.album_id
INNER JOIN artist AS ar ON ar.artist_id = al.artist_id
WHERE il.invoice_id = 4;

 * sqlite:///chinook.db
Done.


track_id,track_name,artist_name,track_type,unit_price,quantity
3448,"Lamentations of Jeremiah, First Set \ Incipit Lamentatio",The King's Singers,Protected AAC audio file,0.99,1
2560,Violent Pornography,System Of A Down,MPEG audio file,0.99,1
3336,War Pigs,Cake,Purchased AAC audio file,0.99,1
829,Let's Get Rocked,Def Leppard,MPEG audio file,0.99,1
1872,Attitude,Metallica,MPEG audio file,0.99,1
748,Dealer,Deep Purple,MPEG audio file,0.99,1
1778,You're What's Happening (In The World Today),Marvin Gaye,MPEG audio file,0.99,1
2514,Spoonman,Soundgarden,MPEG audio file,0.99,1


# 4. Combining Multiple Joins with Subqueries

Because the invoice_line table contains each individual song from each customer purchase, it contains information about which songs are purchased the most. We can use the table to find out which artists are purchased the most. Specifically, what we want to produce is a query that lists the top 10 artists, calculated by the number of times a track by that artist has been purchased.

We'll need to use a GROUP BY clause to get the number of tracks purchased from each artist, but before we do we'll have to join a few tables. As we start thinking about how we will write this query, we can look at the database schema from the first screen of this lesson and think about what tables we will need to include.

Writing our query would be a lot easier if we had one table that contained both the track.track_id and the artist.name columns. We can write a subquery that creates this table for us, and then use that subquery to calculate our answer. This means our process will be:

* Write a subquery that produces a table with track.track_id and artist.name,
* Join that subquery to the invoice_line table,
* Use a GROUP BY statement to calculate the number of times each artist has had a track purchased, and find the top 10.

In [8]:
%%sql

SELECT
    t.track_id,
    ar.name artist_name
FROM track t
INNER JOIN album al ON al.album_id = t.album_id
INNER JOIN artist ar ON ar.artist_id = al.artist_id LIMIT 10;

 * sqlite:///chinook.db
Done.


track_id,artist_name
1,AC/DC
6,AC/DC
7,AC/DC
8,AC/DC
9,AC/DC
10,AC/DC
11,AC/DC
12,AC/DC
13,AC/DC
14,AC/DC


In [14]:
%%sql

SELECT ta.album_title album,
        ta.artist_name artist,
        COUNT(*) tracks_purchased
FROM invoice_line il
INNER JOIN (
            SELECT
                t.track_id,
                al.title album_title,
                ar.name artist_name
            FROM track t
            INNER JOIN album al ON al.album_id = t.album_id
            INNER JOIN artist ar ON ar.artist_id = al.artist_id
            ) ta
            ON ta.track_id = il.track_id
GROUP BY 1,2
ORDER BY 3 DESC LIMIT 5;

 * sqlite:///chinook.db
Done.


album,artist,tracks_purchased
Are You Experienced?,Jimi Hendrix,187
Faceless,Godsmack,96
Mezmerize,System Of A Down,93
Get Born,JET,90
The Doors,The Doors,83


# 5. Recursive Joins